In [19]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils,svm
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import json
import matplotlib.pyplot as plt
import multiprocessing
from sklearn.metrics import accuracy_score, f1_score
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
import jieba
import langdetect
import hanzidentifier
import cantoseg
from nltk.tokenize import word_tokenize
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve

In [399]:
from json import JSONDecoder
from functools import partial
aa=[]

def json_parse(fileobj, decoder=JSONDecoder(), buffersize=50000):
    buffer = ''
    for chunk in iter(partial(fileobj.read, buffersize), ''):
         buffer += chunk
         while buffer:
             try:
                 result, index = decoder.raw_decode(buffer)
                 yield result
                 buffer = buffer[index:].lstrip()
             except ValueError:
                 # Not enough data to decode, read more
                 break
                
with open('yelp_academic_dataset_review.json', 'r') as infh:
    for data in json_parse(infh):
        aa.append(data)
        #print(data)

In [121]:
print(aa[1])

{'review_id': 'UmFMZ8PyXZTY2QcwzsfQYA', 'user_id': 'nIJD_7ZXHq-FX8byPMOkMQ', 'business_id': 'lbrU8StCq3yDfr-QMnGrmQ', 'stars': 1.0, 'useful': 1, 'funny': 1, 'cool': 0, 'text': 'I am actually horrified this place is still in business. My 3 year old son needed a haircut this past summer and the lure of the $7 kids cut signs got me in the door. We had to wait a few minutes as both stylists were working on people. The decor in this place is total garbage. It is so tacky. The sofa they had at the time was a pleather sofa with giant holes in it. And my son noticed ants crawling all over the floor and the furniture. It was disgusting and I should have walked out then. Actually, I should have turned around and walked out upon entering but I didn\'t. So the older black male stylist finishes the haircut he was doing and it\'s our turn. I tell him I want a #2 clipper around the back and sides and then hand cut the top into a standard boys cut. Really freaking simple, right? WRONG! Rather than use

In [400]:
len(aa)

108449

In [401]:
aa_df = pd.DataFrame(columns = ['star','text'])

In [402]:
aa_df

,star,text


In [403]:
for i in range(len(aa)):
    #aa_df = aa_df.append([{'star':aa[i]['stars']}],[{'text':aa[i]['text']}])#, ignore_index=True
    #aa_df[i]={'star':aa[i]['stars'],{'text':aa[i]['text']}
           
    aa_df=aa_df.append(pd.DataFrame({'star':[aa[i]['stars']],'text':[aa[i]['text']]}))
    #aa[i]['stars']
    #aa[i]['text']

In [231]:
aa_df.index=range(len(aa_df))

In [404]:
aa_df.shape

(108449, 2)

In [233]:
aa_df.head(10)

,star,text
0,2.0,"As someone who has worked with many museums, I..."
1,1.0,I am actually horrified this place is still in...
2,5.0,I love Deagan's. I do. I really do. The atmosp...
3,1.0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g..."
4,4.0,"Oh happy day, finally have a Canes near my cas..."
5,5.0,This is definitely my favorite fast food sub s...
6,5.0,"Really good place with simple decor, amazing f..."
7,5.0,"Awesome office and staff, very professional an..."
8,5.0,Most delicious authentic Italian I've had in t...
9,4.0,I have been here twice. Very nice and laid bac...


In [405]:
#search eng symptom
def search_symptom_words(x):
    collect_word = 0
    """
    old_symptom_words = ['肚屙', '拉肚子', '屙肚', '兜屙', '屙啡啡', '屙瀡瀡', '肚瀉','屙肚仔', '拉稀', '又屙又呕', '嘔心', '痙攣', '嘔吐',
                     '腹瀉', '肚子疼', '肚子痛', '胃痛', '病菌', '病痛', '病徵', '病入膏肓', '生病', 'sick', '肠胃炎', 'got sick', 
                     'stomachache']
    
    new_symptom_words = ['昏厥','中毒','腸胃炎','發燒','腹痛','瀉肚','瀉腹','泄瀉','下痢','嘔嘔','潰瘍','肚柯','柯水',
                     '發炎','过敏','頭暈','抽筋','肠炎','溃疡','腹泻','肚泻','泻肚','发炎','肚痛','痾'] ## no 过敏 发炎 抽筋 中毒 昏厥
    ## add  呕左 又柯又呕 急症室 疴咗 厠所 医院 疴左 肚疴 清肠胃 又疴又呕 泻咗 柯左 左呕 上吐下泻 肚疴 发烧 落肚 大泻特泻
    
    symptom_words=['bacteria','virus','parasite','toxin','allergens','allergen','physical hazard','food poisoning','abdominal cramp','abdominal pain','nausea',
                  'watery diarrhea','diarrhea','severe diarrhea','fever','slight fever','vomit','blurred vision','difficulty swallowing','double vision',
                   'muscle weakness','intense abdominal cramp','fatigue','loss of appetite','substantial weight loss','dark urine','headache','jaundice',
                   'muscle ache','decreased apptite','right upper-quadrant abdominal pain','left lower-quadrant pain','chronic diarrhea','occasional bloody diarrhea','weight loss',
                  'hematochezia','melena','weight loss','anorexia','upset stomach','stomach cramp','dropping eyelids','slurred speech','diffculty swallowing','breathing and dry mouth',
                  'paralysis','loss of balance','joint ache','back ache','chest pain','gastroenteritis','stomach flu','food poisoning','foodborne illness','foodborne disease',
                  'clinic','microorganisms','gastroenteritis','germ','pathogen','illness','poison','symptom','disease','doctor','infect','throwing up', 'throw up']# ,'sick'
   #,'chill','chills' number decrease from 7200 to 870 after remove chill
   """
    symptom_words=['food poisoning','nausea',
                   'diarrhea','vomit','foodborne illness','foodborne disease','poison','throwing up', 'throw up']# ,'sick'
    #,'chill','chills' number decrease from 7200 to 870 after remove chill
    collect_word = 0
    for en in symptom_words:
        if en in str(x):
            collect_word = en
    return collect_word

aa_df['symptom'] = aa_df['text'].apply(search_symptom_words)
eng_symptom=aa_df[aa_df['symptom']!=0]
eng_symptom=eng_symptom.reset_index(drop=True)



In [406]:
eng_symptom.shape

(332, 3)

In [603]:
#11,19,32,55
#need to exclude,not restaurant 1,2, 3 ,4,5,6,7,8,9,10,12,13,14
#new id :18,22,28, 34?, 35, 36,38,42,48,49,54,58,59,61,62,67,70,74,76,78,79,88,89,90,96,99,103,105,108,109,114,116
#118,121,135,139,141
#new-third file 143,144,151,152,157,158,159,163,164,166,167,169,171,181,182,184,198,204,205,207,213,214,218,221,225
#229,232,236,238,239,241,242,245,251,254,255,257,259,261,262,264,267,274,275,278,279,280,283,285,287,291,292,293,295
#296,297,301,307,310,311,312,316,317,319,321,328,331
n=331




print(eng_symptom['text'].tolist()[n])

What an absolute waste !!! My wife got food poisoning the first day. First meal upon arrival, no alcohol, 6 hours later she became violently ill. Was sick the whole trip. Room was filthy, the sink backed up, the hot water ran out on one day, The bed was gross. If they just simply move the bed when cleaning and did a thorough job it could be better. Stay at motel 6........it would be an upgrade !!!!  Managers take no responsibility for my wife's illness so far, Just kick the can down the road. Oh and the 30 bucks to check in early...WTH...... if the rooms open give the customer the key. Just another way to rip you off.


In [453]:
real_symptom_yelp

,star,text,symptom
11,1.0,"Hmm.. I never write reviews on here, but I fel...",poison
19,2.0,"sadly I have to write a bad review, I got food...",poison
32,1.0,The b7 I ordered looks/tastes nothing like nig...,diarrhea
55,2.0,The food was pretty good! Went there for dinne...,diarrhea


In [382]:
real_symptom_yelp_2=eng_symptom.loc[[18,22,28, 34,35,36,38,42,48,49,54,58,59,61,62,67,70,74,76,78,79,88,89,90,96,99,103,105,108,109,114,116,
                                     118,121,135,139,141]]

In [604]:
real_symptom_yelp_3 = eng_symptom.loc[[143,144,151,152,157,158,159,163,164,166,167,169,171,181,182,184,198,204,205,207,213,214,218,221,225
,229,232,236,238,239,241,242,245,251,254,255,257,259,261,262,264,267,274,275,278,279,280,283,285,287,291,292,293,295
,296,297,301,307,310,311,312,316,317,319,321,328,331]]

In [605]:
real_symptom_yelp_3.shape

(67, 3)

In [609]:
real_symptom_yelp_final =pd.concat([real_symptom_yelp,real_symptom_yelp_2])

In [611]:
real_symptom_yelp_final =pd.concat([real_symptom_yelp_final,real_symptom_yelp_3])

In [614]:
real_symptom_yelp_final.shape

(108, 3)

In [615]:
real_symptom_yelp_final.index=range(108)

In [617]:
real_symptom_yelp_final.to_csv("real_symptom_yelp_final.csv")

In [27]:
############################Doc2Vec model part ####################################################

In [1298]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=40)) for doc in sents])
    #targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    #print("targets:",targets)
    #print("regressors:",regressors)
    return targets, regressors


def get_vectors(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=40)) for doc in sents])
    return targets, regressors

In [1299]:
def train_dbow(tagged_data):  
    """
    cores = multiprocessing.cpu_count()

    model_dbow = Doc2Vec(dm=0, vector_size=200, negative=5, hs=0, min_count=2, sample = 0, workers=cores, epoches = 30)
    model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

    for epoch in range(30):
        model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha
    model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
    
    return model_dbow
    #model_dbow.save('model_dbow')
    """
    max_epochs = 40
    model = Doc2Vec(dm=0,vector_size=5)
    #model = Doc2Vec(dm=0,vector_size=5)  # of course, if non-default parameters needed, use them here
                       # but most users won't need to change alpha/min_alpha at all
    #model = Doc2Vec(dm=0,min_count=1,window=5,sample=1e-3,negative=5,workers=3,vector_size=5)
    model.build_vocab([x for x in tqdm(train_tagged.values)])
    model.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=model.corpus_count, epochs=max_epochs)
    return model





In [1300]:
def train_dmm(train_tagged):
    """
    model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065,epoches = 30)
    model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])
    
    
    for epoch in range(30):
        model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
        model_dmm.alpha -= 0.002
        model_dmm.min_alpha = model_dmm.alpha
        
    model_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
    return model_dmm

    #model_dmm.save('model_dmm')
    """
    max_epochs = 40
    model = Doc2Vec(dm=1,vector_size=5)  # of course, if non-default parameters needed, use them here
                       # but most users won't need to change alpha/min_alpha at all

    model.build_vocab([x for x in tqdm(train_tagged.values)])
    model.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=model.corpus_count, epochs=max_epochs)
    return model

In [20]:
def concat_model(model_dbow,model_dmm):   
    new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])
    return new_model
    

In [21]:
############################Logistic Regression part ####################################################

In [1225]:
def logreg_weight(X_train,y_train,X_test,y_test):
    logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=10000,class_weight='balanced')#??? 
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
 

    print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
    print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
    print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
    print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')
    #return y_pred

In [1226]:
def logreg(X_train,y_train,X_test,y_test):
    logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=10000)#??? 
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
 

    print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
    print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
    print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
    print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')
    #return y_pred

In [23]:
############################Text Preprocessing #################################################

In [37]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 1:
                continue
            tokens.append(word.lower())
    return tokens

In [972]:
##### read from csv
df = pd.read_csv('all_real_fake_symptom_train.csv')
#df = df[['cut_review','label']]
df = df[['cut_review','label','sympt_count','m_usr_score_hygiene','m_usr_score_value','score_hygiene_diff','score_value_diff']]
df = df[pd.notnull(df['cut_review'])]
df = df[pd.notnull(df['label'])]
df['label']=df['label'].astype('int')

df_test = pd.read_csv('all_real_fake_symptom_test.csv')
df_test = df_test[['cut_review','label','sympt_count','m_usr_score_hygiene','m_usr_score_value','score_hygiene_diff','score_value_diff']]
#df_test = df_test[pd.notnull(df['cut_review'])]
#df_test = df_test[pd.notnull(df['label'])]

df.index=range((df.shape)[0])
df_test.index=range((df_test.shape)[0])

In [973]:
df.head(2)

,cut_review,label,sympt_count,m_usr_score_hygiene,m_usr_score_value,score_hygiene_diff,score_value_diff
0,又 来到 我们 最爱 的 平价 饭堂 哈哈 这次 点 了 两款 没有 吃 过 的 菜式 试试...,0,3.0,3.617021,3.659574,-2.617021,0.340426
1,见 佢 张 poster 写 26810 再有 85 折 咁 平 既 自助 晚餐 一 於 同...,0,3.0,3.000000,3.000000,-2.000000,-2.000000


In [ ]:
X_train_new=[]
y_train_new=[]
y_test_new=[]
X_test_new=[]


for i in range(len(df)):
    aa=(np.append([],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))

    X_train_new.append(aa)
    y_train_new.append(float(df.loc[i]['label']))
X_train_new=tuple(X_train_new)
y_train_new=tuple(y_train_new)


for i in range(len(df_test)):
    bb=(np.append([],float(df_test.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff'])))
    X_test_new.append(bb)
    y_test_new.appned(float(df_test.loc[i]['label']))
    
X_test_new=tuple(X_test_new)
y_test_new=tuple(y_test_new)


In [674]:
data = pd.read_csv('all_real_fake_symptom_train.csv')

In [675]:
data.head(2)

,review_id,rtr_id,usr_id,usr_name,usr_num_cmt,usr_date_publish,usr_score_taste,usr_score_decor,usr_score_service,usr_score_hygiene,...,language,clean_rev_cmt,cut_review,label,sympt_count,tend_count,m_usr_score_hygiene,m_usr_score_value,score_hygiene_diff,score_value_diff
0,2233930,37556,98626,VIANNAKIT,222,2012-05-04,4,3,4,1,...,s_cn,又来到我们最爱的平价饭堂哈哈这次点了两款没有吃过的菜式试试看第一款卡邦尼杂菌烩饭浓浓的卡邦尼...,又 来到 我们 最爱 的 平价 饭堂 哈哈 这次 点 了 两款 没有 吃 过 的 菜式 试试...,0.0,3.0,0.0,3.617021,3.659574,-2.617021,0.340426
1,2282733,211,593313,water姐,2,2013-02-05,1,1,3,1,...,s_cn,见佢张poster写26810再有85折咁平既自助晚餐一於同朋友试试其实咁既价钱预左唔会有太...,见 佢 张 poster 写 26810 再有 85 折 咁 平 既 自助 晚餐 一 於 同...,0.0,3.0,3.0,3.000000,3.000000,-2.000000,-2.000000


In [545]:
####read in english sympotom reviews
en_r = pd.read_csv('eng_real_symptom_final.csv')


en_f = pd.read_csv('eng_fake_symptom_final.csv')

"""
en_r = en_r[['cut_review']]
en_r = en_r[pd.notnull(en_r['cut_review'])]
#en_r = en_r[pd.notnull(en_r['label'])]

en_f = pd.read_csv('eng_fake_symptom_final.csv')
en_f = en_f[['cut_review']]
en_f = en_f[pd.notnull(en_f['cut_review'])]
#en_f = en_f[pd.notnull(en_f['label'])]

"""

"\nen_r = en_r[['cut_review']]\nen_r = en_r[pd.notnull(en_r['cut_review'])]\n#en_r = en_r[pd.notnull(en_r['label'])]\n\nen_f = pd.read_csv('eng_fake_symptom_final.csv')\nen_f = en_f[['cut_review']]\nen_f = en_f[pd.notnull(en_f['cut_review'])]\n#en_f = en_f[pd.notnull(en_f['label'])]\n\n"

In [546]:
en_f.head(2)

,review_id,rtr_id,usr_id,usr_name,usr_num_cmt,usr_date_publish,usr_score_taste,usr_score_decor,usr_score_service,usr_score_hygiene,usr_score_value,usr_date_visit,usr_eat_way,usr_per_price,rev_cmt,symptom
0,1966200,90,102827,foodchick,4,2009-01-12,5,3,4,3,4,None,None,$25,for someone living on hk side the biggest head...,headache
1,2166891,35459,24346,tabero,275,2011-05-03,4,4,4,4,3,None,堂食,$500,It is a pre mother day buffet ....even there d...,sick


In [547]:
en_r['clean_rev_cmt'] = en_r['rev_cmt'].apply(eng_remove_punctuation)
en_r['cut_review'] = en_r['clean_rev_cmt'].apply(lambda x: " ".join([w for w in list(word_tokenize(x))]))
en_r['label']= 0

en_f['clean_rev_cmt'] = en_f['rev_cmt'].apply(eng_remove_punctuation)
en_f['cut_review'] = en_f['clean_rev_cmt'].apply(lambda x: " ".join([w for w in list(word_tokenize(x))]))
en_f['label']= 1

In [548]:
en_f.head(2)

,review_id,rtr_id,usr_id,usr_name,usr_num_cmt,usr_date_publish,usr_score_taste,usr_score_decor,usr_score_service,usr_score_hygiene,usr_score_value,usr_date_visit,usr_eat_way,usr_per_price,rev_cmt,symptom,clean_rev_cmt,cut_review,label
0,1966200,90,102827,foodchick,4,2009-01-12,5,3,4,3,4,None,None,$25,for someone living on hk side the biggest head...,headache,for someone living on hk side the biggest head...,for someone living on hk side the biggest head...,1
1,2166891,35459,24346,tabero,275,2011-05-03,4,4,4,4,3,None,堂食,$500,It is a pre mother day buffet ....even there d...,sick,It is a pre mother day buffet even there dont ...,It is a pre mother day buffet even there dont ...,1


In [555]:
en_r = en_r[['cut_review','label']]
en_r = en_r[pd.notnull(en_r['cut_review'])]
en_r = en_r[pd.notnull(en_r['label'])]

en_f = en_f[['cut_review','label']]

In [556]:
en_r.shape

(24, 2)

In [557]:
en_f.shape

(267, 2)

In [562]:
en_r_train = en_r.head(12)
en_r_test = en_r.tail(12)
en_f_train = en_f.head(134)
en_f_test = en_f.tail(134)

In [568]:
df_en = df.append(en_r_train)
df_en=df_en.append(en_f_train)
df_test_en = df_test.append(en_r_test)
df_test_en=df_test_en.append(en_f_test)

In [505]:
df_0=df.loc[df['label']==0]
df_1=df.loc[df['label']==1]

In [335]:
all_cn_symptom = df.append(df_test)
all_cn_symptom.index=range((all_cn_symptom.shape)[0])
all_cn_symptom['cut_review'].apply(lambda x: len(x.split(' '))).sum()
all_cn_symptom_tagged = all_cn_symptom.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['cut_review']), tags=[r.label]), axis=1)

In [679]:
df['cut_review'].apply(lambda x: len(x.split(' '))).sum()
df_test['cut_review'].apply(lambda x: len(x.split(' '))).sum()

49659

In [680]:
df.head(2)

,cut_review,label,sympt_count,m_usr_score_hygiene,m_usr_score_value,score_hygiene_diff,score_value_diff
0,又 来到 我们 最爱 的 平价 饭堂 哈哈 这次 点 了 两款 没有 吃 过 的 菜式 试试...,0,3.0,3.617021,3.659574,-2.617021,0.340426
1,见 佢 张 poster 写 26810 再有 85 折 咁 平 既 自助 晚餐 一 於 同...,0,3.0,3.000000,3.000000,-2.000000,-2.000000


In [573]:
###for two csv data with eng reviews
df_en['cut_review'].apply(lambda x: len(x.split(' '))).sum()
df_test_en['cut_review'].apply(lambda x: len(x.split(' '))).sum()

df_en_tagged = df_en.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['cut_review']), tags=[r.label]), axis=1)
df_test_en_tagged = df_test_en.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['cut_review']), tags=[r.label]), axis=1)

In [681]:
df_tagged = df.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['cut_review']), tags=[r.label]), axis=1)

In [682]:
df_tagged.values[0]

TaggedDocument(words=['又', '来到', '我们', '最爱', '的', '平价', '饭堂', '哈哈', '这次', '点', '了', '两款', '没有', '吃', '过', '的', '菜式', '试试看', '第一款', '卡', '邦尼', '杂菌', '烩饭', '浓浓的', '卡', '邦尼', '汁', '有', '浓浓的', '芝士', '和', '奶', '油香', '配上', '杂菌', '淡淡', '又', '浓浓的', '酱汁', '包住', '每颗', '饭粒', '不会', '很乾湿', '湿滑', '滑', '的', '口感', '真的', '不错', '第二款', '铁板', '羊', '扒', '两件', '小小的', '羊架', '闻起来', '很香', '可是', '吃', '起来', '味道', '很', 'so', '肉质', '很', '熟', '而且', '好多', '油', '喔', '不', '好吃', '整体', '上', '是', '可以', '接受', '的', '但', '最', '可恶', '的', '是', '第二天', '起来', '就', '开始', '肚子痛', '我们', '两人', '都', '肚泻', '了', '真的', '不', '知道', '是', '那个', '菜式', '出', '了', '味道', '最近'], tags=[0])

In [509]:
train_tagged = df.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['cut_review']), tags=[r.label]), axis=1)
test_tagged = df_test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['cut_review']), tags=[r.label]), axis=1)
#train_tagged.values[0]

In [910]:
####dbow
model_dbow=train_dbow(train_tagged)
model_dbow.save('model_dbow')
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3153052.82it/s]


Testing accuracy 0.8210526315789474
Testing F1 score: 0.8177977785505349
Area Under Curve: 0.7653490328006729
Recall score: [0.63414634 0.89655172]
Testing accuracy 0.8175438596491228
Testing F1 score: 0.8161531878476679
Area Under Curve: 0.770154992190316
Recall score: [0.65853659 0.8817734 ]


In [912]:
####dmm
model_dmm=train_dmm(train_tagged)
model_dmm.save('model_dmm')
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3227629.98it/s]


Testing accuracy 0.7473684210526316
Testing F1 score: 0.7540011805882668
Area Under Curve: 0.7245284152348912
Recall score: [0.67073171 0.77832512]
Testing accuracy 0.7473684210526316
Testing F1 score: 0.7540011805882668
Area Under Curve: 0.7245284152348912
Recall score: [0.67073171 0.77832512]


In [913]:
####dbow+dmm
new_model=concat_model(model_dbow,model_dmm)
y_train, X_train = get_vectors(new_model, train_tagged)
y_test, X_test = get_vectors(new_model, test_tagged)
logreg(X_train,y_train,X_test,y_test)
print("-----")
logreg_weight(X_train,y_train,X_test,y_test)

Testing accuracy 0.7789473684210526
Testing F1 score: 0.7863148685361697
Area Under Curve: 0.7757719572269615
Recall score: [0.76829268 0.78325123]
-----
Testing accuracy 0.775438596491228
Testing F1 score: 0.7831209602954754
Area Under Curve: 0.7733089030397694
Recall score: [0.76829268 0.77832512]


In [918]:
X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*100)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*100)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*100)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df.loc[i]['sympt_count']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_hygiene']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_value']*100)))
    bb=(np.append(bb,float(df.loc[i]['score_hygiene_diff']*100)))        
    bb=(np.append(bb,float(df.loc[i]['score_value_diff']*100)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)

logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

Testing accuracy 0.8350877192982457
Testing F1 score: 0.833497874972516
Area Under Curve: 0.7897392767031118
Recall score: [0.68292683 0.89655172]
----
Testing accuracy 0.8421052631578947
Testing F1 score: 0.8452921668902493
Area Under Curve: 0.8310104529616725
Recall score: [0.80487805 0.85714286]
============end=============


/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [919]:
X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*100)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*100)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*100)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df.loc[i]['sympt_count']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_hygiene']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_value']*100)))
    bb=(np.append(bb,float(df.loc[i]['score_hygiene_diff']*100)))        
    bb=(np.append(bb,float(df.loc[i]['score_value_diff']*100)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)

logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Testing accuracy 0.7157894736842105
Testing F1 score: 0.727376704775117
Area Under Curve: 0.7168989547038328
Recall score: [0.7195122  0.71428571]
----
Testing accuracy 0.7192982456140351
Testing F1 score: 0.7309056424845898
Area Under Curve: 0.7229965156794426
Recall score: [0.73170732 0.71428571]
============end=============


/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [917]:
X_train_new=[]
X_test_new=[]

y_train, X_train = get_vectors(new_model, train_tagged)
y_test, X_test = get_vectors(new_model, test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df.loc[i]['score_value_diff'])))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)

logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")



Testing accuracy 0.7368421052631579
Testing F1 score: 0.7464915776887416
Area Under Curve: 0.7316772798269854
Recall score: [0.7195122  0.74384236]
----
Testing accuracy 0.7368421052631579
Testing F1 score: 0.7464915776887416
Area Under Curve: 0.7316772798269854
Recall score: [0.7195122  0.74384236]
============end=============


In [922]:
####dbow--only chinese-vector size=25
model_dbow_chi_25=train_dbow(train_tagged)
model_dbow_chi_25.save('model_dbow_chi_25')
y_train, X_train = vec_for_learning(model_dbow_chi_25, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_25, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3195986.54it/s]


Testing accuracy 0.8491228070175438
Testing F1 score: 0.8482704582227334
Area Under Curve: 0.8104950138171333
Recall score: [0.7195122  0.90147783]
Testing accuracy 0.8385964912280702
Testing F1 score: 0.8435183220147142
Area Under Curve: 0.8430854259281509
Recall score: [0.85365854 0.83251232]


In [1031]:
model_dbow_chi_10=train_dbow(train_tagged)
model_dbow_chi_10.save('model_dbow_chi_10')
y_train, X_train = vec_for_learning(model_dbow_chi_10, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_10, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3153052.82it/s]


Testing accuracy 0.8456140350877193
Testing F1 score: 0.8471805985552116
Area Under Curve: 0.8225699867836117
Recall score: [0.76829268 0.87684729]
Testing accuracy 0.8315789473684211
Testing F1 score: 0.8378822687339098
Area Under Curve: 0.8490628379190195
Recall score: [0.8902439  0.80788177]


In [1034]:
model_dbow_chi_4=train_dbow(train_tagged)
model_dbow_chi_4.save('model_dbow_chi_4')
y_train, X_train = vec_for_learning(model_dbow_chi_4, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_4, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3049808.37it/s]


Testing accuracy 0.856140350877193
Testing F1 score: 0.85352370432494
Area Under Curve: 0.8081521086146821
Recall score: [0.69512195 0.92118227]
Testing accuracy 0.8491228070175438
Testing F1 score: 0.8529113976408019
Area Under Curve: 0.8468400817013095
Recall score: [0.84146341 0.85221675]


In [1140]:
model_dbow_chi_5=train_dbow(train_tagged)
model_dbow_chi_5.save('model_dbow_chi_5')
y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3222312.63it/s]


Testing accuracy 0.8701754385964913
Testing F1 score: 0.869936693378734
Area Under Curve: 0.8398113660939565
Recall score: [0.76829268 0.91133005]
Testing accuracy 0.8631578947368421
Testing F1 score: 0.8665940583253784
Area Under Curve: 0.8639613120269133
Recall score: [0.86585366 0.86206897]


In [1301]:
model_dbow_chi_5_a=train_dbow(train_tagged)
model_dbow_chi_5_a.save('model_dbow_chi_5_a')
y_train, X_train = vec_for_learning(model_dbow_chi_5_a, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5_a, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3258096.22it/s]


Testing accuracy 0.8596491228070176
Testing F1 score: 0.8596491228070176
Area Under Curve: 0.8287876967439626
Recall score: [0.75609756 0.90147783]
Testing accuracy 0.8631578947368421
Testing F1 score: 0.8671925514030778
Area Under Curve: 0.8712303256037487
Recall score: [0.8902439  0.85221675]


In [1176]:
model_dmm_chi_5=train_dmm(train_tagged)
model_dmm_chi_5.save('model_dmm_chi_5')
y_train, X_train = vec_for_learning(model_dmm_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dmm_chi_5, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3153052.82it/s]


Testing accuracy 0.775438596491228
Testing F1 score: 0.7624573060558373
Area Under Curve: 0.6824462333293284
Recall score: [0.46341463 0.90147783]
Testing accuracy 0.7649122807017544
Testing F1 score: 0.7702529695769235
Area Under Curve: 0.7404781929592695
Recall score: [0.68292683 0.79802956]


In [1177]:
model_dbow_dmm_chi_5 = concat_model(model_dbow_chi_5,model_dmm_chi_5)

y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

print("---------")
clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

Testing accuracy 0.8385964912280702
Testing F1 score: 0.8352960635855373
Area Under Curve: 0.7849333173134686
Recall score: [0.65853659 0.91133005]
Testing accuracy 0.8491228070175438
Testing F1 score: 0.8504140872773945
Area Under Curve: 0.8250330409708039
Recall score: [0.76829268 0.8817734 ]
---------
Testing accuracy 0.8105263157894737
Testing F1 score: 0.8028682782168656
Area Under Curve: 0.7361528295085906
Recall score: [0.56097561 0.91133005]


In [1179]:

X_train_new=[]
X_test_new=[]


y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

#y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
#y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10000)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10000)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10000)))
    aa=normalize(aa)
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10000)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10000)))
    bb=normalize(bb)
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

#90%
print("------ RF:________")
from sklearn.neighbors import KNeighborsClassifier
RF=KNeighborsClassifier(n_neighbors=4,algorithm='brute',metric='cosine')

RF.fit(X_train,y_train)

y_pred = RF.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

Testing accuracy 0.8842105263157894
Testing F1 score: 0.8839975913918439
Area Under Curve: 0.8569325964195603
Recall score: [0.79268293 0.92118227]
----
Testing accuracy 0.8912280701754386
Testing F1 score: 0.8936991351773825
Area Under Curve: 0.8945692658897033
Recall score: [0.90243902 0.88669951]
============end=============
Testing accuracy 0.8175438596491228
Testing F1 score: 0.8111402497533702
Area Under Curve: 0.7483479514598101
Recall score: [0.58536585 0.91133005]
------ RF:________
Testing accuracy 0.7824561403508772
Testing F1 score: 0.7909081335883636
Area Under Curve: 0.7927730385678241
Recall score: [0.81707317 0.76847291]


In [1036]:
model_dbow_chi_6=train_dbow(train_tagged)
model_dbow_chi_6.save('model_dbow_chi_6')
y_train, X_train = vec_for_learning(model_dbow_chi_6, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_6, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3056162.13it/s]


Testing accuracy 0.856140350877193
Testing F1 score: 0.8547534654015564
Area Under Curve: 0.8154211221915175
Recall score: [0.7195122  0.91133005]
Testing accuracy 0.856140350877193
Testing F1 score: 0.8600770363236001
Area Under Curve: 0.8590352036525291
Recall score: [0.86585366 0.85221675]


In [1038]:
model_dbow_chi_7=train_dbow(train_tagged)
model_dbow_chi_7.save('model_dbow_chi_7')
y_train, X_train = vec_for_learning(model_dbow_chi_7, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_7, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3076995.96it/s]


Testing accuracy 0.8526315789473684
Testing F1 score: 0.8531545672005375
Area Under Curve: 0.8238615883695782
Recall score: [0.75609756 0.89162562]
Testing accuracy 0.8596491228070176
Testing F1 score: 0.8636408547839525
Area Under Curve: 0.8651327646281388
Recall score: [0.87804878 0.85221675]


In [1040]:
model_dbow_chi_8=train_dbow(train_tagged)
model_dbow_chi_8.save('model_dbow_chi_8')
y_train, X_train = vec_for_learning(model_dbow_chi_8, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_8, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 2933915.65it/s]


Testing accuracy 0.8315789473684211
Testing F1 score: 0.8321766482291856
Area Under Curve: 0.7981797428811727
Recall score: [0.7195122  0.87684729]
Testing accuracy 0.8491228070175438
Testing F1 score: 0.8532515259003611
Area Under Curve: 0.8504745884897271
Recall score: [0.85365854 0.84729064]


In [1027]:
model_dbow_chi_50=train_dbow(train_tagged)
model_dbow_chi_50.save('model_dbow_chi_50')
y_train, X_train = vec_for_learning(model_dbow_chi_50, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_50, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3091586.56it/s]


Testing accuracy 0.8140350877192982
Testing F1 score: 0.8114653469472017
Area Under Curve: 0.7604229244262886
Recall score: [0.63414634 0.88669951]
Testing accuracy 0.8210526315789474
Testing F1 score: 0.8246644558089492
Area Under Curve: 0.8053286074732668
Recall score: [0.76829268 0.84236453]


In [1097]:
##test vector size=7
X_train_new=[]
X_test_new=[]


#y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
#y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

y_train, X_train = vec_for_learning(model_dbow_chi_7, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_7,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10000)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10000)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10000)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10000)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10000)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")




Testing accuracy 0.8807017543859649
Testing F1 score: 0.8793052743694064
Area Under Curve: 0.8435660218671152
Recall score: [0.75609756 0.93103448]
----
Testing accuracy 0.9192982456140351
Testing F1 score: 0.9209271075216832
Area Under Curve: 0.925177219752493
Recall score: [0.93902439 0.91133005]
============end=============


In [1144]:
##test vector size=7
X_train_new=[]
X_test_new=[]


#y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
#y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

y_train, X_train = vec_for_learning(model_dbow_chi_7, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_7,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff'])))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")




Testing accuracy 0.8807017543859649
Testing F1 score: 0.8826235739121402
Area Under Curve: 0.8726420761744562
Recall score: [0.85365854 0.89162562]
----
Testing accuracy 0.8736842105263158
Testing F1 score: 0.8780055401662049
Area Under Curve: 0.8931575153189955
Recall score: [0.93902439 0.84729064]
============end=============


In [1141]:

X_train_new=[]
X_test_new=[]


#y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
#y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10000)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10000)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10000)))
    aa=normalize(aa)
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10000)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10000)))
    bb=normalize(bb)
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

#90%
print("------ RF:________")
from sklearn.neighbors import KNeighborsClassifier
RF=KNeighborsClassifier(n_neighbors=4,algorithm='brute',metric='cosine')

RF.fit(X_train,y_train)

y_pred = RF.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

Testing accuracy 0.8842105263157894
Testing F1 score: 0.8839975913918439
Area Under Curve: 0.8569325964195603
Recall score: [0.79268293 0.92118227]
----
Testing accuracy 0.9228070175438596
Testing F1 score: 0.9242630688396117
Area Under Curve: 0.9276402739396852
Recall score: [0.93902439 0.91625616]
============end=============
Testing accuracy 0.8526315789473684
Testing F1 score: 0.850906515397502
Area Under Curve: 0.8093235612159075
Recall score: [0.70731707 0.91133005]
------ RF:________
Testing accuracy 0.8140350877192982
Testing F1 score: 0.8198876297430162
Area Under Curve: 0.8149405262525532
Recall score: [0.81707317 0.81280788]


In [1302]:
##for weiwei check 
X_train_new=[]
X_test_new=[]


#y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
#y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

y_train, X_train = vec_for_learning(model_dbow_chi_5_a, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5_a,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10000)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10000)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10000)))
    aa=normalize(aa)
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10000)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10000)))
    bb=normalize(bb)
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

#90%
print("------ RF:________")
from sklearn.neighbors import KNeighborsClassifier
RF=KNeighborsClassifier(n_neighbors=4,algorithm='brute',metric='cosine')

RF.fit(X_train,y_train)

y_pred = RF.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

Testing accuracy 0.8842105263157894
Testing F1 score: 0.8839975913918439
Area Under Curve: 0.8569325964195603
Recall score: [0.79268293 0.92118227]
----
Testing accuracy 0.9122807017543859
Testing F1 score: 0.9142734961107922
Area Under Curve: 0.9202511113781088
Recall score: [0.93902439 0.90147783]
============end=============
Testing accuracy 0.8596491228070176
Testing F1 score: 0.8596491228070176
Area Under Curve: 0.8287876967439626
Recall score: [0.75609756 0.90147783]
------ RF:________
Testing accuracy 0.8210526315789474
Testing F1 score: 0.8270167983388041
Area Under Curve: 0.8271356482037726
Recall score: [0.84146341 0.81280788]


In [1142]:

X_train_new=[]
X_test_new=[]


#y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
#y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff'])))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')


Testing accuracy 0.887719298245614
Testing F1 score: 0.8884973628100101
Area Under Curve: 0.8702991709720052
Recall score: [0.82926829 0.91133005]
----
Testing accuracy 0.9052631578947369
Testing F1 score: 0.9080563817405921
Area Under Curve: 0.92259401658056
Recall score: [0.96341463 0.8817734 ]
============end=============
Testing accuracy 0.856140350877193
Testing F1 score: 0.8553276462123738
Area Under Curve: 0.8190556289799351
Recall score: [0.73170732 0.90640394]


In [1145]:

X_train_new=[]
X_test_new=[]


#y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
#y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    aa=normalize(aa)
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff'])))
    bb=normalize(bb)
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')


Testing accuracy 0.8666666666666667
Testing F1 score: 0.8639402264402265
Area Under Curve: 0.8191757779646761
Recall score: [0.70731707 0.93103448]
----
Testing accuracy 0.9052631578947369
Testing F1 score: 0.9074153757996557
Area Under Curve: 0.9116904962153071
Recall score: [0.92682927 0.89655172]
============end=============
Testing accuracy 0.8526315789473684
Testing F1 score: 0.850906515397502
Area Under Curve: 0.8093235612159075
Recall score: [0.70731707 0.91133005]


In [1146]:

X_train_new=[]
X_test_new=[]


#y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
#y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10000)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10000)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10000)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10000)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10000)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')


Testing accuracy 0.8842105263157894
Testing F1 score: 0.883556398170935
Area Under Curve: 0.8532980896311426
Recall score: [0.7804878  0.92610837]
----
Testing accuracy 0.9157894736842105
Testing F1 score: 0.9173778932795764
Area Under Curve: 0.9190796587768834
Recall score: [0.92682927 0.91133005]
============end=============
Testing accuracy 0.8526315789473684
Testing F1 score: 0.850906515397502
Area Under Curve: 0.8093235612159075
Recall score: [0.70731707 0.91133005]


In [1128]:

X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow_chi_50, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_50,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff'])))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


Testing accuracy 0.8280701754385965
Testing F1 score: 0.8295416343393567
Area Under Curve: 0.7993511954823982
Recall score: [0.73170732 0.86699507]
----
Testing accuracy 0.8210526315789474
Testing F1 score: 0.8259494842074051
Area Under Curve: 0.8162321278385198
Recall score: [0.80487805 0.82758621]
============end=============


In [944]:
#try to use SVM model
#test_data_label = pd.to_numeric(all_real_fake_symptom_test['label']).tolist()
y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5, test_tagged)


clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

Testing accuracy 0.8701754385964913
Testing F1 score: 0.869936693378734
Area Under Curve: 0.8398113660939565
Recall score: [0.76829268 0.91133005]


In [945]:
#try to use SVM model
#test_data_label = pd.to_numeric(all_real_fake_symptom_test['label']).tolist()
y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5, test_tagged)


clf = svm.SVC(kernel='linear')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

Testing accuracy 0.8385964912280702
Testing F1 score: 0.8329317593972121
Area Under Curve: 0.7740297969482158
Recall score: [0.62195122 0.92610837]


In [950]:
#try to use SVM model
#test_data_label = pd.to_numeric(all_real_fake_symptom_test['label']).tolist()
y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5, test_tagged)


clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

Testing accuracy 0.8666666666666667
Testing F1 score: 0.866170418784798
Area Under Curve: 0.8337138051183467
Recall score: [0.75609756 0.91133005]


In [1082]:
def normalize(x):
    """
        argument
            - x: input image data in numpy array [32, 32, 3]
        return
            - normalized x 
    """
    mean_val=np.mean(x)
    """
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    """
    x=x-mean_val
    return x

In [985]:

X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10000)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10000)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10000)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10000)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10000)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10000)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10000)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

# vector size=5 and weight =10000 has the best performance now


Testing accuracy 0.8912280701754386
Testing F1 score: 0.8906135861605752
Area Under Curve: 0.8618587047939444
Recall score: [0.79268293 0.93103448]
----
Testing accuracy 0.9157894736842105
Testing F1 score: 0.9178047683310842
Area Under Curve: 0.9263486723537186
Recall score: [0.95121951 0.90147783]
============end=============


In [1061]:

X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*10010)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10010)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10010)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10010)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10010)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10010)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10010)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10010)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10010)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10010)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

# vector size=5 and weight =10000 has the best performance now


Testing accuracy 0.8912280701754386
Testing F1 score: 0.8906135861605752
Area Under Curve: 0.8618587047939444
Recall score: [0.79268293 0.93103448]
----
Testing accuracy 0.9087719298245615
Testing F1 score: 0.9107304778800934
Area Under Curve: 0.9141535504024992
Recall score: [0.92682927 0.90147783]
============end=============


In [1066]:

X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*10005)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10005)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10005)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10005)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10005)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10005)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10005)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10005)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10005)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10005)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

# vector size=5 and weight =10005 has the best performance now

clf = svm.SVC(kernel='rbf')
clf.fit(X_train_new,y_train)

y_pred = clf.predict(X_test_new)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

Testing accuracy 0.8912280701754386
Testing F1 score: 0.8906135861605752
Area Under Curve: 0.8618587047939444
Recall score: [0.79268293 0.93103448]
----
Testing accuracy 0.9157894736842105
Testing F1 score: 0.9175973641970093
Area Under Curve: 0.922714165565301
Recall score: [0.93902439 0.90640394]
============end=============
Testing accuracy 0.8982456140350877
Testing F1 score: 0.8997390171750046
Area Under Curve: 0.8922263606872521
Recall score: [0.87804878 0.90640394]


In [1101]:
#try to normalize the input feature

from sklearn.preprocessing import StandardScaler

X_train_new=[]
X_test_new=[]

#y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
#y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)
y_train, X_train = get_vectors(model_dbow_dmm_chi_5, train_tagged)
y_test, X_test = get_vectors(model_dbow_dmm_chi_5, test_tagged)

#y_train, X_train = vec_for_learning(model_dbow_chi_7, train_tagged)
#y_test, X_test = vec_for_learning(model_dbow_chi_7,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    """
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*10005)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*10005)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*10005)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*10005)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*10005)))
    """
    aa=normalize(aa)
    #scaler = StandardScaler().fit(aa)
    #aa=scaler.transform(aa)
    X_train_new.append(aa)
    
#X_train_new=np.array(X_train_new)
#X_train_new_scaler=StandardScaler().fit(X_train_new)
#X_train_new=X_train_new_scaler.transform(X_train_new)
X_train_new=tuple(X_train_new)


for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff'])))
    """
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count']*10005)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene']*10005)))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value']*10005)))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff']*10005)))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff']*10005)))
    """
    #bb=scaler.transform(bb)
    bb=normalize(bb)
    X_test_new.append(bb)
    #
    
    #bb=scaler.fit(bb)
#X_test_new=np.array(X_test_new)
#X_test_new_scaler=StandardScaler().fit(X_test_new)
#X_test_new=X_test_new_scaler.transform(X_test_new)
X_test_new=tuple(X_test_new)





logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


print("==========svm===============")
clf = svm.SVC(kernel='rbf')
clf.fit(X_train_new,y_train)


y_pred = clf.predict(X_test_new)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')





Testing accuracy 0.8385964912280702
Testing F1 score: 0.8337543859649122
Area Under Curve: 0.7776643037366335
Recall score: [0.63414634 0.92118227]
----
Testing accuracy 0.8491228070175438
Testing F1 score: 0.8508874525495025
Area Under Curve: 0.8286675477592215
Recall score: [0.7804878  0.87684729]
============end=============
==========svm===============
Testing accuracy 0.8982456140350877
Testing F1 score: 0.898780837176372
Area Under Curve: 0.8813228403219993
Recall score: [0.84146341 0.92118227]


#using the original normalization method can help svm achieve accuracy about 91.7% (without *10000)
#using the second normalization method can help weighted logistics regression achieve accuracy about 91.7%(without *10000)
#standerscaler do not have significant imporvement 


In [1086]:
#try to normalize the input feature

from sklearn.preprocessing import StandardScaler

X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    #aa=normalize(aa)
    #scaler = StandardScaler().fit(aa)
    #aa=scaler.transform(aa)
    X_train_new.append(aa)
    
X_train_new=np.array(X_train_new)
X_train_new_scaler=StandardScaler().fit(X_train_new)
X_train_new=X_train_new_scaler.transform(X_train_new)
X_train_new=tuple(X_train_new)


for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff'])))
    #bb=scaler.transform(bb)
    #bb=normalize(bb)
    X_test_new.append(bb)
    #
    
    #bb=scaler.fit(bb)
X_test_new=np.array(X_test_new)
X_test_new_scaler=StandardScaler().fit(X_test_new)
X_test_new=X_test_new_scaler.transform(X_test_new)
X_test_new=tuple(X_test_new)





logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


print("==========svm===============")
clf = svm.SVC(kernel='rbf')
clf.fit(X_train_new,y_train)


y_pred = clf.predict(X_test_new)
 
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')





Testing accuracy 0.8912280701754386
Testing F1 score: 0.8931320706830367
Area Under Curve: 0.8873002523128679
Recall score: [0.87804878 0.89655172]
----
Testing accuracy 0.8771929824561403
Testing F1 score: 0.8812860380756499
Area Under Curve: 0.8956205695061876
Recall score: [0.93902439 0.85221675]
============end=============
==========svm===============
Testing accuracy 0.8912280701754386
Testing F1 score: 0.8918002052575011
Area Under Curve: 0.8727622251591973
Recall score: [0.82926829 0.91625616]


In [1077]:
X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow_chi_5, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_5,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df_test.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df_test.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df_test.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df_test.loc[i]['score_value_diff'])))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")


Testing accuracy 0.9017543859649123
Testing F1 score: 0.9021030448003583
Area Under Curve: 0.8837858945091914
Recall score: [0.84146341 0.92610837]
----
Testing accuracy 0.9017543859649123
Testing F1 score: 0.9038635915631776
Area Under Curve: 0.9055929352396973
Recall score: [0.91463415 0.89655172]
============end=============


In [937]:
model_dbow_chi_1=train_dbow(train_tagged)
model_dbow_chi_1.save('model_dbow_chi_1')
y_train, X_train = vec_for_learning(model_dbow_chi_1, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_1, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3124510.81it/s]


Testing accuracy 0.7157894736842105
Testing F1 score: 0.6377633672463892
Area Under Curve: 0.5315391084945332
Recall score: [0.09756098 0.96551724]
Testing accuracy 0.5824561403508772
Testing F1 score: 0.6021945184875839
Area Under Curve: 0.5978613480716088
Recall score: [0.63414634 0.56157635]


In [939]:
X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow_chi_1, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_1,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene'])))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value'])))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff'])))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff'])))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df.loc[i]['sympt_count'])))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_hygiene'])))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_value'])))
    bb=(np.append(bb,float(df.loc[i]['score_hygiene_diff'])))        
    bb=(np.append(bb,float(df.loc[i]['score_value_diff'])))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

Testing accuracy 0.43157894736842106
Testing F1 score: 0.44142477195752905
Area Under Curve: 0.5028535383876006
Recall score: [0.67073171 0.33497537]
----
Testing accuracy 0.3508771929824561
Testing F1 score: 0.31125043941132996
Area Under Curve: 0.48618286675477596
Recall score: [0.80487805 0.16748768]
============end=============


In [923]:
####dmm--only chinese-vector size=25
model_dmm_chi_25=train_dmm(train_tagged)
model_dmm_chi_25.save('model_dmm_chi_25')
y_train, X_train = vec_for_learning(model_dmm_chi_25, train_tagged)
y_test, X_test = vec_for_learning(model_dmm_chi_25, test_tagged)
logreg(X_train,y_train,X_test,y_test)
logreg_weight(X_train,y_train,X_test,y_test)

100%|██████████| 1399/1399 [00:00<00:00, 3059348.96it/s]


Testing accuracy 0.7508771929824561
Testing F1 score: 0.7521875668800833
Area Under Curve: 0.70154992190316
Recall score: [0.58536585 0.81773399]
Testing accuracy 0.7263157894736842
Testing F1 score: 0.736135477582846
Area Under Curve: 0.7170191036885738
Recall score: [0.69512195 0.73891626]


In [925]:
####dbow+dmm--new model-chi vector size=25
new_model_chi_25=concat_model(model_dbow_chi_25,model_dmm_chi_25)
y_train, X_train = get_vectors(new_model_chi_25, train_tagged)
y_test, X_test = get_vectors(new_model_chi_25, test_tagged)
logreg(X_train,y_train,X_test,y_test)
print("-----")
logreg_weight(X_train,y_train,X_test,y_test)

Testing accuracy 0.8105263157894737
Testing F1 score: 0.8111987292578339
Area Under Curve: 0.772497897392767
Recall score: [0.68292683 0.86206897]
-----
Testing accuracy 0.8070175438596491
Testing F1 score: 0.8098498601594915
Area Under Curve: 0.7809383635708278
Recall score: [0.7195122  0.84236453]


In [927]:
X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dbow_chi_25, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_chi_25,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*100)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*100)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*100)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df.loc[i]['sympt_count']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_hygiene']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_value']*100)))
    bb=(np.append(bb,float(df.loc[i]['score_hygiene_diff']*100)))        
    bb=(np.append(bb,float(df.loc[i]['score_value_diff']*100)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)

logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

Testing accuracy 0.8385964912280702
Testing F1 score: 0.8385964912280702
Area Under Curve: 0.8031058512555569
Recall score: [0.7195122  0.88669951]
----
Testing accuracy 0.8105263157894737
Testing F1 score: 0.8166683278319449
Area Under Curve: 0.812477472065361
Recall score: [0.81707317 0.80788177]
============end=============


In [894]:
X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dmm_chi_25, train_tagged)
y_test, X_test = vec_for_learning(model_dmm_chi_25,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*100)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*100)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*100)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df.loc[i]['sympt_count']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_hygiene']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_value']*100)))
    bb=(np.append(bb,float(df.loc[i]['score_hygiene_diff']*100)))        
    bb=(np.append(bb,float(df.loc[i]['score_value_diff']*100)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)

logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

Testing accuracy 0.6526315789473685
Testing F1 score: 0.6688792731911665
Area Under Curve: 0.668929472545957
Recall score: [0.70731707 0.63054187]
----
Testing accuracy 0.6350877192982456
Testing F1 score: 0.6523381180223287
Area Under Curve: 0.6566142016099964
Recall score: [0.70731707 0.60591133]
============end=============


In [896]:
X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(new_model_chi_25, train_tagged)
y_test, X_test = vec_for_learning(new_model_chi_25,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*100)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*100)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*100)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df.loc[i]['sympt_count']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_hygiene']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_value']*100)))
    bb=(np.append(bb,float(df.loc[i]['score_hygiene_diff']*100)))        
    bb=(np.append(bb,float(df.loc[i]['score_value_diff']*100)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)


logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Testing accuracy 0.656140350877193
Testing F1 score: 0.6716125737207449
Area Under Curve: 0.6604890063678962
Recall score: [0.67073171 0.65024631]
----
Testing accuracy 0.6280701754385964
Testing F1 score: 0.6452415956091345
Area Under Curve: 0.633515559293524
Recall score: [0.64634146 0.62068966]
============end=============


/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
################TEXT

In [51]:
### real symptom from train data
real_sentences = ['第二天起来就开始肚子痛', '我们两人都肚泻了', '即刻肚痛又肚屙屙左3次',
                  '一路走一路肚痛火都黎番到屋企要食喇x丸到今日个肚仲系唔','5点几就开始肚痛肚all到依家所以唔怕有肠胃炎都可以去试下既一碗小面',
                 '食完同行既有一个发烧一个肚屙一个肚屙加发烧','我都顶你地唔顶走人返到屋企仲要肚屙试问自己要求亦不算高','最惨系食完肠胃炎',
                 '同埋我地有3个人食完都胃痛兼肚屙','其他食左鳗鱼饭嘅都肚屙到想死','我自己亦都开始发烧肚屙想呕','第二日同友人相继出现肚屙',
                 '我同朋友一齐食这个一齐肠胃炎了','第二日就肠胃炎','食完又屙又呕','重点系有位朋友食完即刻肚屙','而其他4-5位朋友食完都有唔舒服同肠胃炎',
                 '本人怀疑食完鱼生寿司肚屙','但返到屋企即刻肚屙','食完仲要肚屙','仲要肚屙','我同男朋友都有d肚屙','当晚即刻肚屙',
                 '因为人生第一次肠胃炎就系食呢间餐厅嗰包','吃完胃痛左几个','肚子痛了一晚上','朋友还拉肚子了','食完系个个都有屙','行返屋企半路上开始有啲胃痛',
                 '胃痛想呕朋友吃完晚上回家','个个都出现肚屙肚泻','我仲发埋烧及呕左','但食完我同我男朋友都系屋企肚屙左成晚','朋友食完即时肚屙',
                 '返到屋企仲要肠胃炎','肚屙又心跳快','食完我番到公司肚屙','我朋友下午还拉肚子了','医生话我系肠胃炎重发埋烧添','重点系我22号零晨胃痛痛醒了',
                 '食完会胃痛','食到佢有胃痛','但回家和友人都肚屙','我在酒店呕咗三次','仲要朋友食完既晚上肚痛添','我今朝屙左3次','仲要食到肚痛',
                 '食完返到去已经呕左2次','而且最重要的是我最近两次来吃我都拉肚子了','我同朋友食完都肚泻','朋友仲搞到肠胃炎','朋友吃了当晚肚子痛',
                 '同行友人讲返原来佢都肚屙','我果晚半夜肚痛痛醒左','但系第二朝我同女友都轻微肚泻','跟住个肚好肚痛','食完肚疼','最惨系食完一小时就肚痛',
                 '野食唔热都算食完同行几个朋友都肚柯隔左好耐比多次机会佢点知','因为都系一次但点解要上黎写劣平系因为我同我老公食完都肚痛','食完肚痛及疴咗两次',
                 '无奈食完无耐行商场既时候竟然肚柯','部份亲友咁岩有人当晚肚痛','最夸张是回家肚痛要去厕所','第二日起身肚痛','食甘耐嘢未试过甘样肚痛',
                 '但味道太一般般当晚返到屋企肚柯都冇谂过关餐饭事直到第二日奶奶话柯左几次','有人返到屋企就肚痛','食完仲有小小肚痛','食完夜晚反到屋企肚柯',
                 '全部人就食物中毒又痀又呕又发烧','男朋友食完表示肚痛','食完我女朋友寻日就呕左成晚','第二日就发烧','值得推介但食完有点肚痛赠送左',
                 '但无人会想食完肚柯架','友人肚泻了很多次','胃抽筋屙呕了很多次','点知个一晚就已经肚痛','食完个个都肚痛','佢食完仲觉得有少少肚痛',
                 '返到屋企既肚痛','肚痛要去厕所','结果都系食完肚痛','不嗜辣的朋友回到家也肚子痛','但系我反完屋企就搞肚痛','但系食完未走已经肚屙',
                 '我就出现肠胃炎症状','竟然发现同朋友一样有肚屙既情况','只是之後那天就开始胃痛','我同同行朋友加埋肚屙左五次','但系返到屋企之後当晚就即刻搞肚痛',
                 '就夜晚开始肚屙','3个人之後很快肚子痛','我返到屋企亦痾左水数次','一人肚泻呕吐','另一人也出现肚泻','我老婆食完仲肚痛','跟住嗰两日肠胃炎',
                  '食完冇几耐就肚痛','去到黄昏我地两个都肚痛','有成10个同事都有肚痾嘅情况出现','今次食完同女朋友都肚胀又痾','但食完佢碗粉就会肚痛',
                  '两个人肚痾同呕左几日','不过我朋友食完话有阵怪味重肚痛']

fake_sentences = ['希望有改善其实我都怕食生野肚痛','我怕肚痛','食完都无肚痛','起码到而家都冇肚痛','从来未试过肚痛','唔怕食到肚痛','好容易肚痛','食咁多次都无肚泻',
                 '因为他很易肚痛','食过咁多次都无肚痛','免得肚柯','起码我次次食完都无试过肚痛','其实我哋有惊食到肚痛㗎','觉得会肚痛','猪大肠清得几乾净唔怕食到肚痛',
                 '否则食完肚痛就无谓啦','原本会觉得好似争先咁唔系咁新鲜惊食咗会肚痛','容易肚痛','但最重要都系食左冇肚泻','但可能是怕未煮至全煮易令客人肚痛吧',
                 '可能系担心人食咁生会肚痛','返到屋企无肚痛','最惊食完沙律会肚痛','食完无肚痛','不过食完冇肚痛','食呢间既蚝冇肚痛','食完无肚痛','总之食完唔肚痛就算',
                 '其他生腌蟹我又怕肚痛','担心会肚痛','食左咁多次都无肚痛','因为怕食完肚痛所以就冇食','一阵食到人肚痛就唔好','真系食左惊肚痛','今晚唔好肚痛',
                 '吾惊肚痛啦叫左韭菜猪红','如果我今晚肚痛我实告硬佢','随手拾返又吾洗手就拿熟食希望食完吾肚痛啦','未到75度希望唔会肚柯啦','但可以过下蚝瘾食完又吾会肚痛',
                 '无肚屙无肚痛','喎返到屋企都冇肚痛','也从不肚痛','食完都惊肚痛','唔怕食到肚痛','玻璃肠既我第二日都冇肚痛','无咁易肚痛呀','害怕我食到肚痛',
                 '仲要饮完冇肚痛呢吓好重要','希望今晚唔会肚痛啦','不要吃到肚痛就','不过我怕会肚痛','开头我都惊好易肚痛','食咁多次都冇肚痛','好味食完系无屙同肚痛嫁今次',
                 '唔怕会食到肚痛','有点刺激又唔会肚痛','食完都惊肚痛','所以唔会整到我肚痛','食完第2日唔肚痛我已经好感恩','好彩第二日无肚痛','我食完两日都冇肚柯',
                 '好彩之後冇肚痛','唔会怕食左会肚痛','最重要系唔会肚痾','是要看看有冇肚泻现象','完全无肚痛','我们之後都冇肚痛的现象','兼要有事後肚痛的心理准备',
                 '唔知佢会唔会肚痛','弟日都没有肚痛','都唔会肚痛','暂时无事无肚痛','惊肚痛都冇饮','平时钟意食蚝既我因为好怕食到唔乾净既蚝会肚痛','不过起码食完无肚痛',
                 '但系又惊肚痛','惊会肚痛','呢晚食咗两只结果系无肚痛','所以食完无肚痛','下价刺身食完又惊肚泻','而且一路食一落觉得自己会肚痾','食完都冇肚痛等等问题出现',
                 '唔知会唔会肚痾','吃完没肚痛','老实讲一路食一路惊今晚痾','好怕会肚痾','未吃光看已经有吃後会肚痛的感觉','好大机会会死到人肚痾','唔乾净我惊食左肚痛']

len(fake_sentences)

90

In [55]:
real_test_sen=pd.DataFrame(real_sentences,columns=['review'])
fake_test_sen=pd.DataFrame(fake_sentences,columns=['review'])

real_test_sen['review']=real_test_sen['review'].apply(lambda x: " ".join([w for w in list(jieba.cut(x))]))
real_test_sen['label']=0
fake_test_sen['review']=fake_test_sen['review'].apply(lambda x: " ".join([w for w in list(jieba.cut(x))]))
fake_test_sen['label']=1
real_test_sen.head(10)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.583 seconds.
Prefix dict has been built successfully.


,review,label
0,第二天 起来 就 开始 肚子痛,0
1,我们 两人 都 肚泻 了,0
2,即刻 肚痛 又 肚 屙 屙 左 3 次,0
3,一路 走 一路 肚痛 火 都 黎番 到 屋企 要食 喇 x 丸 到 今日 个 肚仲系 唔,0
4,5 点 几 就 开始 肚痛 肚 all 到 依家 所以 唔 怕 有 肠胃炎 都 可以 去 试...,0
5,食完 同行 既有 一个 发烧 一个 肚 屙 一个 肚 屙 加 发烧,0
6,我 都 顶 你 地 唔 顶 走 人 返到 屋企 仲要 肚 屙 试问 自己 要求 亦 不算 高,0
7,最惨 系食 完 肠胃炎,0
8,同埋 我 地 有 3 个人 食完 都 胃痛 兼肚 屙,0
9,其他 食 左 鳗鱼 饭 嘅 都 肚 屙 到 想 死,0


In [66]:
second_test=pd.concat([real_test_sen,fake_test_sen],ignore_index=True)
second_test.index=range((second_test.shape)[0])
second_test['review'].apply(lambda x: len(x.split(' '))).sum()
second_tagged=second_test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.label]), axis=1)


In [67]:
##using second text to test the model trained by the whole real and fake data 
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, second_tagged)
logreg(X_train,y_train,X_test,y_test)

Testing accuracy 0.5408163265306123
Testing F1 score: 0.3796458980943371


In [68]:
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, second_tagged)
logreg(X_train,y_train,X_test,y_test)

Testing accuracy 0.5663265306122449
Testing F1 score: 0.4681078006115338


In [69]:
new_model=concat_model(model_dbow,model_dmm)
y_train, X_train = get_vectors(new_model, train_tagged)
y_test, X_test = get_vectors(new_model, second_tagged)
logreg(X_train,y_train,X_test,y_test)

Testing accuracy 0.5969387755102041
Testing F1 score: 0.5110660496367756


In [72]:
###using second text for training and testing
sec_train, sec_test = train_test_split(second_test, test_size=0.3, random_state=42)
sec_train.index=range((sec_train.shape)[0])
sec_test.index=range((sec_test.shape)[0])
sec_train['review'].apply(lambda x: len(x.split(' '))).sum()
sec_test['review'].apply(lambda x: len(x.split(' '))).sum()
sec_train_tagged=sec_train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.label]), axis=1)
sec_test_tagged=sec_test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['review']), tags=[r.label]), axis=1)


In [76]:
####second-dbow
model_dbow_sec=train_dbow(sec_train_tagged)
model_dbow_sec.save('model_dbow_sec')
y_train_sec, X_train_sec = vec_for_learning(model_dbow_sec, sec_train_tagged)
y_test_sec, X_test_sec = vec_for_learning(model_dbow_sec, sec_test_tagged)
logreg(X_train_sec,y_train_sec,X_test_sec,y_test_sec)

100%|██████████| 137/137 [00:00<00:00, 477258.84it/s]


Testing accuracy 0.576271186440678
Testing F1 score: 0.576271186440678


In [119]:
#####second-dbow-test "all_real_fake"
y_train_sec, X_train_sec = vec_for_learning(model_dbow_sec, sec_train_tagged)
y_test, X_test = get_vectors(model_dbow_sec, test_tagged)
logreg(X_train_sec,y_train_sec,X_test,y_test)

Testing accuracy 0.45964912280701753
Testing F1 score: 0.47758373840652585


In [113]:
####second-dmm
model_dmm_sec=train_dmm(sec_train_tagged)
model_dmm_sec.save('model_dmm_sec')
y_train_sec, X_train_sec = vec_for_learning(model_dmm_sec, sec_train_tagged)
y_test_sec, X_test_sec = vec_for_learning(model_dmm_sec, sec_test_tagged)
y_pred=logreg(X_train_sec,y_train_sec,X_test_sec,y_test_sec)

100%|██████████| 137/137 [00:00<00:00, 415187.61it/s]


Testing accuracy 0.8135593220338984
Testing F1 score: 0.8129136400322841


In [120]:
#####second-dmm-test "all_real_fake"
y_train_sec, X_train_sec = vec_for_learning(model_dmm_sec, sec_train_tagged)
y_test, X_test = get_vectors(model_dmm_sec, test_tagged)
logreg(X_train_sec,y_train_sec,X_test,y_test)

Testing accuracy 0.4842105263157895
Testing F1 score: 0.4937460337462026


In [114]:
index_list=[]
for i in range(0,len(y_pred)):
    if y_pred[i] != y_test_sec[i]:
        index_list.append(i)
    else:
        continue
false_det=[]
for j in range(0,len(index_list)):
    false_det.append(sec_test_tagged[index_list[j]])
    

In [115]:
false_det

[TaggedDocument(words=['但', '无人', '会想食', '完肚', '柯架'], tags=[0]),
 TaggedDocument(words=['不要', '吃', '到', '肚痛', '就'], tags=[1]),
 TaggedDocument(words=['其他', '食', '左', '鳗鱼', '饭', '嘅', '都', '肚', '屙', '到', '想', '死'], tags=[0]),
 TaggedDocument(words=['好容易', '肚痛'], tags=[1]),
 TaggedDocument(words=['最惊食', '完', '沙律', '会肚痛'], tags=[1]),
 TaggedDocument(words=['食', '完仲有', '小小', '肚痛'], tags=[0]),
 TaggedDocument(words=['觉得', '会肚痛'], tags=[1]),
 TaggedDocument(words=['惊会', '肚痛'], tags=[1]),
 TaggedDocument(words=['食甘耐', '嘢', '未试过', '甘样', '肚痛'], tags=[0]),
 TaggedDocument(words=['有点', '刺激', '又', '唔', '会', '肚痛'], tags=[1]),
 TaggedDocument(words=['最', '重要', '系', '唔', '会', '肚', '痾'], tags=[1])]

In [102]:
new_model_sec=concat_model(model_dbow_sec,model_dmm_sec)
y_train_new, X_train_new = get_vectors(new_model_sec, sec_train_tagged)
y_test_new, X_test_new = get_vectors(new_model_sec, sec_test_tagged)
y_pred=logreg(X_train_new,y_train_new,X_test_new,y_test_new)

Testing accuracy 0.847457627118644
Testing F1 score: 0.847457627118644


In [92]:
index_list=[]
for i in range(0,len(y_pred)):
    if y_pred[i] != y_test_new[i]:
        index_list.append(i)
    else:
        continue
false_det=[]
for j in range(0,len(index_list)):
    false_det.append(sec_test_tagged[index_list[j]])
    

In [98]:
false_det

[TaggedDocument(words=['食', '咁', '多次', '都', '无肚泻'], tags=[1]),
 TaggedDocument(words=['但', '无人', '会想食', '完肚', '柯架'], tags=[0]),
 TaggedDocument(words=['不要', '吃', '到', '肚痛', '就'], tags=[1]),
 TaggedDocument(words=['而且', '一路', '食一落', '觉得', '自己', '会肚', '痾'], tags=[1]),
 TaggedDocument(words=['我们', '之後都', '冇', '肚痛', '的', '现象'], tags=[1]),
 TaggedDocument(words=['食', '完仲有', '小小', '肚痛'], tags=[0]),
 TaggedDocument(words=['一人', '肚泻', '呕吐'], tags=[0]),
 TaggedDocument(words=['食完', '肚疼'], tags=[0]),
 TaggedDocument(words=['食甘耐', '嘢', '未试过', '甘样', '肚痛'], tags=[0]),
 TaggedDocument(words=['食完', '第', '2', '日', '唔', '肚痛', '我', '已经', '好', '感恩'], tags=[1]),
 TaggedDocument(words=['但', '味道', '太', '一般般', '当晚', '返到', '屋企', '肚柯', '都', '冇', '谂', '过关', '餐饭', '事', '直到', '第二日', '奶奶', '话', '柯', '左', '几次'], tags=[0]),
 TaggedDocument(words=['但', '最', '重要', '都', '系食', '左', '冇', '肚泻'], tags=[1])]

In [132]:
######using all data to train Doc2Vec model:
#lr = LogisticRegression(penalty='l2',solver='newton-cg',multi_class='multinomial')

kl_review = pd.read_csv('kl_reviews_cn.csv',encoding='utf-8',low_memory=False)
hki_review = pd.read_csv('hki_reviews_cn.csv',encoding='utf-8',low_memory=False)
nt_review = pd.read_csv('nt_reviews_cn.csv',encoding='utf-8',low_memory=False)
oi_review = pd.read_csv('oi_reviews_cn.csv',encoding='utf-8',low_memory=False)

all_review = pd.concat([kl_review,hki_review,nt_review,oi_review])


In [ ]:

all_review.dropna(subset=['rev_cmt'])
## detect the language function
def language_detect(s):
    if bool(re.match('^(?=.*[a-zA-Z])', s))  == False:
        language_output='NA'
    else:
        language = langdetect.detect(s)
        if language == 'en':
            language_output = 'en'
        else:
            if hanzidentifier.is_simplified(s):
                language_output = 's_cn'
            else:
                language_output = 't_cn'
    return language_output

all_review['language'] = all_review['rev_cmt'].apply(language_detect)

### segment the dataframe according to the language
all_review_en = all_review.loc[all_review['language'] == 'en']
all_review_s_cn = all_review.loc[(all_review['language'] == 's_cn')]
all_review_t_cn = all_review.loc[(all_review['language'] == 't_cn')]

In [161]:
#### 定义删除除字母,数字，汉字以外的所有符号的function
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
"""
#### simplified chinese
#### clean comments
all_review_s_cn['clean_rev_cmt'] = all_review_s_cn['rev_cmt'].apply(remove_punctuation)
all_review_s_cn['clean_rev_cmt'] = all_review_s_cn['rev_cmt'].apply(remove_punctuation)
#### segment word
all_review_s_cn['cut_review'] = all_review_s_cn['clean_rev_cmt'].apply(lambda x: " ".join([w for w in list(jieba.cut(x))]))
all_review_s_cn['cut_review'] = all_review_s_cn['clean_rev_cmt'].apply(lambda x: " ".join([w for w in list(jieba.cut(x))]))
"""
#### cantonese
#### clean comments
all_review_t_cn['clean_rev_cmt'] = all_review_t_cn['rev_cmt'].apply(remove_punctuation)
all_review_t_cn['clean_rev_cmt'] = all_review_t_cn['rev_cmt'].apply(remove_punctuation)
#### segment word
all_review_t_cn['cut_review'] = all_review_t_cn['clean_rev_cmt'].apply(lambda x: " ".join([w for w in list(cantoseg.cut(x))]))
all_review_t_cn['cut_review'] = all_review_t_cn['clean_rev_cmt'].apply(lambda x: " ".join([w for w in list(cantoseg.cut(x))]))

<ipython-input-161-d4c0f62fe78f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_review_t_cn['clean_rev_cmt'] = all_review_t_cn['rev_cmt'].apply(remove_punctuation)
<ipython-input-161-d4c0f62fe78f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_review_t_cn['clean_rev_cmt'] = all_review_t_cn['rev_cmt'].apply(remove_punctuation)
<ipython-input-161-d4c0f62fe78f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [167]:
all_review_t_cn.shape

(399438, 18)

In [163]:
all_review_s_cn.shape

(361604, 18)

In [164]:
all_review_en.shape

(43643, 16)

In [169]:
all_review_cn=all_review_t_cn.append(all_review_s_cn)

In [174]:
all_review_cn.index=range((all_review_cn.shape)[0])

In [193]:
all_review_cn.head(2)

,review_id,rtr_id,usr_id,usr_name,usr_num_cmt,usr_date_publish,usr_score_taste,usr_score_decor,usr_score_service,usr_score_hygiene,usr_score_value,usr_date_visit,usr_eat_way,usr_per_price,rev_cmt,language,clean_rev_cmt,cut_review
0,100051,4014,20109,美食焚化炉,2.4K,2006-03-18,4,2,4,3,4,None,None,$110,"原本一个二人(6/3)饭局,点知一个下午,变成五人网聚,论地点唔难搵,但门面就唔及红磡店,不...",t_cn,原本一个二人63饭局点知一个下午变成五人网聚论地点唔难搵但门面就唔及红磡店不过笔者食野求好味...,原本 一个 二人 63 饭局 点 知 一个 下午 变成 五 人 网聚 论 地点 唔 难 搵 ...
1,100176,3119,23980,Raymond 仔,307,2006-03-19,5,3,4,3,5,None,None,$40,"20060312 1530久闻这间荣华的上海菜出名,虽然个""朶""唔及另一同名食肆响,但我对佢...",t_cn,200603121530久闻这间荣华的上海菜出名虽然个朶唔及另一同名食肆响但我对佢有兴趣多o...,200603 121 530 久闻 这 间 荣华 的 上海 菜 出名 虽然 个 朶 唔 及 ...


In [177]:
cn_symptom_train = pd.read_csv('all_real_fake_symptom_train.csv')
cn_symptom_test = pd.read_csv('all_real_fake_symptom_test.csv')
cn_symptom = pd.concat([cn_symptom_train,cn_symptom_test], ignore_index=True)
cn_symptom.index=range((cn_symptom.shape)[0])

In [178]:
cn_symptom.shape

(1686, 29)

In [181]:
cn_symptom.head(2)

,review_id,rtr_id,usr_id,usr_name,usr_num_cmt,usr_date_publish,usr_score_taste,usr_score_decor,usr_score_service,usr_score_hygiene,...,language,clean_rev_cmt,cut_review,label,sympt_count,tend_count,m_usr_score_hygiene,m_usr_score_value,score_hygiene_diff,score_value_diff
0,2233930,37556,98626,VIANNAKIT,222,2012-05-04,4,3,4,1,...,s_cn,又来到我们最爱的平价饭堂哈哈这次点了两款没有吃过的菜式试试看第一款卡邦尼杂菌烩饭浓浓的卡邦尼...,又 来到 我们 最爱 的 平价 饭堂 哈哈 这次 点 了 两款 没有 吃 过 的 菜式 试试...,0.0,3.0,0.0,3.617021,3.659574,-2.617021,0.340426
1,2282733,211,593313,water姐,2,2013-02-05,1,1,3,1,...,s_cn,见佢张poster写26810再有85折咁平既自助晚餐一於同朋友试试其实咁既价钱预左唔会有太...,见 佢 张 poster 写 26810 再有 85 折 咁 平 既 自助 晚餐 一 於 同...,0.0,3.0,3.0,3.000000,3.000000,-2.000000,-2.000000


In [189]:
del_id = cn_symptom['usr_id'].values.tolist()

In [197]:

all_review_cn3=all_review_cn[~all_review_cn['usr_id'].isin(del_id)]

In [198]:
all_review_cn3.shape#original:761042

(741042, 18)

In [199]:
##now is the eng data.
real_eng_symptom = pd.read_csv('eng_real_symptom_final.csv')
fake_eng_symptom = pd.read_csv('eng_fake_symptom_final.csv')

In [200]:
eng_symptom = real_eng_symptom.append(fake_eng_symptom)

In [201]:
del_eng_id= eng_symptom['usr_id'].values.tolist()

In [202]:
all_review_en3 = all_review_en[~all_review_en['usr_id'].isin(del_eng_id)]

In [214]:
##need to segment the all_review_en3 and label all three frames 
all_review_en3.index=range((all_review_en3.shape)[0])
all_review_en3.head(2)


,review_id,rtr_id,usr_id,usr_name,usr_num_cmt,usr_date_publish,usr_score_taste,usr_score_decor,usr_score_service,usr_score_hygiene,usr_score_value,usr_date_visit,usr_eat_way,usr_per_price,rev_cmt,language
0,110814,574,32456,zoolook,178,2006-06-11,5,5,4,5,3,None,None,$210,次次黎食都系肉眼扒+凯撒鸡沙拉(虽然d鸡肉好鬼鞋)+molten chocolate cak...,en
1,111547,12751,37105,am418,2,2006-06-17,2,1,1,1,1,None,None,$26,Won't be go there again forever......Went ther...,en


In [297]:
# 对于英文的评论1. remove punctuation(eng) + lower 2. canseg to segment 3. remove the " " 4. label
# 5. send with other reviews to doc2vec 


In [290]:
import string
def eng_remove_punctuation(text):
    remove = str.maketrans('','',string.punctuation) 
    without_punctuation = text.translate(remove)
    return without_punctuation 

In [291]:
#如果中英文掺杂，先去标点符号，转小写，分割，去空格
#整个英文文档，去标点符号，转小写，nltk分词，

all_review_en3['clean_rev_cmt'] = all_review_en3['rev_cmt'].apply(eng_remove_punctuation)
all_review_en3['cut_review'] = all_review_en3['clean_rev_cmt'].apply(lambda x: " ".join([w for w in list(word_tokenize(x))]))

<ipython-input-291-a14e60ad31b1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_review_en3['clean_rev_cmt'] = all_review_en3['rev_cmt'].apply(eng_remove_punctuation)
<ipython-input-291-a14e60ad31b1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_review_en3['cut_review'] = all_review_en3['clean_rev_cmt'].apply(lambda x: " ".join([w for w in list(word_tokenize(x))]))


In [292]:
all_review_en3.shape

(35470, 18)

In [293]:
all_review_en3.head(2)

,review_id,rtr_id,usr_id,usr_name,usr_num_cmt,usr_date_publish,usr_score_taste,usr_score_decor,usr_score_service,usr_score_hygiene,usr_score_value,usr_date_visit,usr_eat_way,usr_per_price,rev_cmt,language,clean_rev_cmt,cut_review
0,110814,574,32456,zoolook,178,2006-06-11,5,5,4,5,3,None,None,$210,次次黎食都系肉眼扒+凯撒鸡沙拉(虽然d鸡肉好鬼鞋)+molten chocolate cak...,en,次次黎食都系肉眼扒凯撒鸡沙拉虽然d鸡肉好鬼鞋molten chocolate cake这是无...,次次黎食都系肉眼扒凯撒鸡沙拉虽然d鸡肉好鬼鞋molten chocolate cake这是无...
1,111547,12751,37105,am418,2,2006-06-17,2,1,1,1,1,None,None,$26,Won't be go there again forever......Went ther...,en,Wont be go there again foreverWent there last ...,Wont be go there again foreverWent there last ...


In [295]:
print(all_review_en3.loc[1]['cut_review'])

Wont be go there again foreverWent there last time i didnt expect its a good restaurant at lease it should fullfill a normal standardHowever this experience its really shit me out1 Very dirty the bench the table and the floor are full of grease with a very disgusting smell2 The food a not worth of this price 3 The taste is lower than normal standard I order 柱侯牛腩饭but it really di


In [298]:
all_review_final3=all_review_cn3.append(all_review_en3)
all_review_final3.index=range((all_review_final3.shape)[0])

In [305]:
all_review_final3['label']=2

In [306]:
all_review_final3.shape

(776512, 19)

In [307]:
all_review_final3.head(2)

,review_id,rtr_id,usr_id,usr_name,usr_num_cmt,usr_date_publish,usr_score_taste,usr_score_decor,usr_score_service,usr_score_hygiene,usr_score_value,usr_date_visit,usr_eat_way,usr_per_price,rev_cmt,language,clean_rev_cmt,cut_review,label
0,100176,3119,23980,Raymond 仔,307,2006-03-19,5,3,4,3,5,None,None,$40,"20060312 1530久闻这间荣华的上海菜出名,虽然个""朶""唔及另一同名食肆响,但我对佢...",t_cn,200603121530久闻这间荣华的上海菜出名虽然个朶唔及另一同名食肆响但我对佢有兴趣多o...,200603 121 530 久闻 这 间 荣华 的 上海 菜 出名 虽然 个 朶 唔 及 ...,2
1,100196,3119,29112,landly,60,2006-03-19,4,4,5,3,4,None,None,$50,再去荣华食饭６个人，叫了两个３人餐有东坡肉，是甜甜的，也不太油腻，挺美味～炒三鲜，是虾，火腿...,t_cn,再去荣华食饭个人叫了两个人餐有东坡肉是甜甜的也不太油腻挺美味炒三鲜是虾火腿和肉时菜狮子头同茶...,再 去 荣华 食饭 个人 叫 了 两个 人餐 有 东坡肉 是 甜甜的 也 不 太 油腻 挺 ...,2


In [308]:
df_3=all_review_final3[['cut_review','label']]
df_3 = df_3[pd.notnull(df_3['cut_review'])]
df_3 = df_3[pd.notnull(df_3['label'])]



In [359]:
df123=df_3.append(df)

In [361]:
df_3['cut_review'].apply(lambda x: len(x.split(' '))).sum()
df123['cut_review'].apply(lambda x: len(x.split(' '))).sum()

122945417

In [313]:
df3_tagged = df123.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['cut_review']), tags=[r.label]), axis=1)

In [362]:
df123_tagged = df123.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['cut_review']), tags=[r.label]), axis=1)

In [325]:
df3_tagged.values[776401]

TaggedDocument(words=['after', 'a', 'leisurely', 'hike', 'a', 'circular', 'route', 'which', 'takes', 'in', 'mo', 'tat', 'wan', 'yung', 'shue', 'ha', 'and', 'back', 'to', 'so', 'kwu', 'wan', 'we', 'decided', 'on', 'lunch', 'at', 'lamma', 'rainbow', 'a', 'long', 'established', 'restaurant', 'on', 'the', 'seafronta', 'good', 'crowd', 'of', 'people', 'eatingeven', 'during', 'lunch', 'with', 'a', 'lively', 'atmosphereyou', 'can', 'choose', 'seafood', 'from', 'the', 'fish', 'tanks', 'cooked', 'to', 'the', 'way', 'you', 'want', 'at', 'market', 'price', 'or', 'choose', 'from', 'the', 'a', 'la', 'carte', 'menu', 'we', 'opted', 'for', 'the', 'latterhot', 'favourites', 'for', 'this', 'place', 'include', 'sweet', 'sour', 'porkjuicy', 'chunks', 'of', 'pork'], tags=[2])

In [326]:
model_dbow_df3=train_dbow(df3_tagged)

100%|██████████| 776512/776512 [00:00<00:00, 3379877.18it/s]


NameError: name 'model_dbow_def' is not defined

In [327]:
model_dbow_df3.save('model_dbow_all_reviews')

In [363]:
model_dbow_df123=train_dbow(df123_tagged)
model_dbow_df123.save('model_dbow_all_reviews_123')

100%|██████████| 777911/777911 [00:00<00:00, 3281021.29it/s]


In [518]:
model_dmm_df123=train_dmm(df123_tagged)
model_dmm_df123.save('model_dmm_all_reviews_123')

100%|██████████| 777911/777911 [00:00<00:00, 3261689.51it/s]


In [521]:
y_train, X_train = vec_for_learning(model_dmm_df123, train_tagged)
y_test, X_test = vec_for_learning(model_dmm_df123,test_tagged)
logreg(X_train,y_train,X_test,y_test)


Testing accuracy 0.712280701754386
Testing F1 score: 0.7142744922006508
Area Under Curve: 0.6562837919019584
Recall score: [0.52439024 0.78817734]


In [580]:
####test the two csv data with eng reviews
y_train, X_train = vec_for_learning(model_dmm_df123, df_en_tagged)
y_test, X_test = vec_for_learning(model_dmm_df123,df_test_en_tagged)
logreg(X_train,y_train,X_test,y_test)



Testing accuracy 0.777262180974478
Testing F1 score: 0.7694847498821139
Area Under Curve: 0.642780478565566
Recall score: [0.40425532 0.88130564]


In [583]:
##test the concat model for both chi reviews and eng reviews
new_model_123=concat_model(model_dbow_df123,model_dmm_df123)

y_train, X_train = get_vectors(new_model_123, df_en_tagged)
y_test, X_test = get_vectors(new_model_123,df_test_en_tagged)
logreg(X_train,y_train,X_test,y_test)

Testing accuracy 0.6960556844547564
Testing F1 score: 0.7115838332380247
Area Under Curve: 0.621535450470358
Recall score: [0.4893617 0.7537092]


In [586]:

y_train, X_train = get_vectors(new_model_123, train_tagged)
y_test, X_test = get_vectors(new_model_123,test_tagged)
logreg(X_train,y_train,X_test,y_test)

Testing accuracy 0.6947368421052632
Testing F1 score: 0.6963425115291163
Area Under Curve: 0.633065000600745
Recall score: [0.48780488 0.77832512]


In [587]:
###training dbow/dmm model using csv data and eng reviews
model_dbow_df_en=train_dbow(df_en_tagged)
model_dbow_df_en.save('model_dbow_df_en')

model_dmm_df_en=train_dmm(df_en_tagged)
model_dmm_df_en.save('model_dmm_df_en')

new_model_en = concat_model(model_dbow_df_en,model_dmm_df_en)

100%|██████████| 1545/1545 [00:00<00:00, 1130727.57it/s]


In [827]:
model_dmm_df_en_5=train_dmm(df_en_tagged)
model_dmm_df_en_5.save('model_dmm_df_en_5')


100%|██████████| 1545/1545 [00:00<00:00, 3100574.01it/s]


In [828]:
y_train, X_train = vec_for_learning(model_dmm_df_en_5, df_en_tagged)
y_test, X_test = vec_for_learning(model_dmm_df_en_2,df_test_en_tagged)

logreg(X_train,y_train,X_test,y_test)
print("----")
logreg_weight(X_train,y_train,X_test,y_test)
print("============end=============")

Testing accuracy 0.5591647331786543
Testing F1 score: 0.5942352538330294
Area Under Curve: 0.49564366437275087
Recall score: [0.38297872 0.60830861]
----
Testing accuracy 0.5104408352668214
Testing F1 score: 0.5511764215166722
Area Under Curve: 0.5450312519729781
Recall score: [0.60638298 0.48367953]
============end=============


In [847]:
#test the result with only chi reviews
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm,test_tagged)

logreg(X_train,y_train,X_test,y_test)
print("----")
logreg_weight(X_train,y_train,X_test,y_test)
print("============end=============")

Testing accuracy 0.7052631578947368
Testing F1 score: 0.7082538699690402
Area Under Curve: 0.6513576835275743
Recall score: [0.52439024 0.77832512]
----
Testing accuracy 0.7017543859649122
Testing F1 score: 0.7126904547139071
Area Under Curve: 0.688874204012976
Recall score: [0.65853659 0.71921182]
============end=============


In [843]:
X_train_new=[]
X_test_new=[]

y_train, X_train = vec_for_learning(model_dmm_df_en_25, train_tagged)
y_test, X_test = vec_for_learning(model_dmm_df_en_25,test_tagged)

for i in range(len(X_train)):
    aa=(np.append(X_train[i],float(df.loc[i]['sympt_count']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_hygiene']*100)))
    aa=(np.append(aa,float(df.loc[i]['m_usr_score_value']*100)))
    aa=(np.append(aa,float(df.loc[i]['score_hygiene_diff']*100)))        
    aa=(np.append(aa,float(df.loc[i]['score_value_diff']*100)))
    X_train_new.append(aa)
X_train_new=tuple(X_train_new)

for i in range(len(X_test)):
    bb=(np.append(X_test[i],float(df.loc[i]['sympt_count']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_hygiene']*100)))
    bb=(np.append(bb,float(df.loc[i]['m_usr_score_value']*100)))
    bb=(np.append(bb,float(df.loc[i]['score_hygiene_diff']*100)))        
    bb=(np.append(bb,float(df.loc[i]['score_value_diff']*100)))
    X_test_new.append(bb)
X_test_new=tuple(X_test_new)

logreg(X_train_new,y_train,X_test_new,y_test)
print("----")
logreg_weight(X_train_new,y_train,X_test_new,y_test)
print("============end=============")

Testing accuracy 0.6526315789473685
Testing F1 score: 0.668748092268783
Area Under Curve: 0.6652949657575393
Recall score: [0.69512195 0.63546798]
----
Testing accuracy 0.6491228070175439
Testing F1 score: 0.6655873833988611
Area Under Curve: 0.6664664183587647
Recall score: [0.70731707 0.62561576]
============end=============


In [786]:
df.loc[0,['sympt_count','m_usr_score_hygiene']]
X_train_new.append(np.append(X_train[1],float(df.loc[1]['m_usr_score_hygiene'])))
X_train_new=tuple(X_train_new)

sympt_count                  3
m_usr_score_hygiene    3.61702
Name: 0, dtype: object

In [708]:
df.head(2)

,cut_review,label,sympt_count,m_usr_score_hygiene,m_usr_score_value,score_hygiene_diff,score_value_diff
0,又 来到 我们 最爱 的 平价 饭堂 哈哈 这次 点 了 两款 没有 吃 过 的 菜式 试试...,0,3.0,3.617021,3.659574,-2.617021,0.340426
1,见 佢 张 poster 写 26810 再有 85 折 咁 平 既 自助 晚餐 一 於 同...,0,3.0,3.000000,3.000000,-2.000000,-2.000000


In [720]:
df.loc[0]['sympt_count']

cut_review             又 来到 我们 最爱 的 平价 饭堂 哈哈 这次 点 了 两款 没有 吃 过 的 菜式 试试...
label                                                                  0
sympt_count                                                            3
m_usr_score_hygiene                                              3.61702
m_usr_score_value                                                3.65957
score_hygiene_diff                                              -2.61702
score_value_diff                                                0.340426
Name: 0, dtype: object

In [610]:
"""
def logreg_multi(X_train,y_train,X_test,y_test):
    
    #logreg = LogisticRegression(penalty='l2',solver='newton-cg',multi_class='multinomial')#???
    logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=10000,multi_class='multinomial')
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
 
    print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
    print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

    
    lr = LogisticRegression(penalty='l2',solver='newton-cg',multi_class='multinomial')
    lr.fit(X_train,y_train)
    
    print("训练集的准确率：%.3f" %lr.score(X_train, y_train))
    print("测试集的准确率：%.3f" %lr.score(X_test, y_test))
    
    y_hat = lr.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_hat) #错误率，也就是np.average(y_test==y_hat)
    print("模型正确率：%.3f" %accuracy)
    return lr 
    """

'\ndef logreg_multi(X_train,y_train,X_test,y_test):\n    \n    #logreg = LogisticRegression(penalty=\'l2\',solver=\'newton-cg\',multi_class=\'multinomial\')#???\n    logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=10000,multi_class=\'multinomial\')\n    logreg.fit(X_train, y_train)\n    y_pred = logreg.predict(X_test)\n \n    print(\'Testing accuracy %s\' % accuracy_score(y_test, y_pred))\n    print(\'Testing F1 score: {}\'.format(f1_score(y_test, y_pred, average=\'weighted\')))\n\n    \n    lr = LogisticRegression(penalty=\'l2\',solver=\'newton-cg\',multi_class=\'multinomial\')\n    lr.fit(X_train,y_train)\n    \n    print("训练集的准确率：%.3f" %lr.score(X_train, y_train))\n    print("测试集的准确率：%.3f" %lr.score(X_test, y_test))\n    \n    y_hat = lr.predict(X_test)\n    accuracy = metrics.accuracy_score(y_test, y_hat) #错误率，也就是np.average(y_test==y_hat)\n    print("模型正确率：%.3f" %accuracy)\n    return lr \n    '

In [517]:
y_train, X_train = vec_for_learning(model_dbow_df123, train_tagged)
y_test, X_test = vec_for_learning(model_dbow_df123,test_tagged)
logreg(X_train,y_train,X_test,y_test)
print("hhh")

Testing accuracy 0.5894736842105263
Testing F1 score: 0.6015776542092333
Area Under Curve: 0.5373663342544756
Recall score: [0.41463415 0.66009852]
hhh


In [486]:
y_df_train,X_df_train = vec_for_learning(model_dbow_df123,df_tagged)

In [470]:
y_test, X_test = vec_for_learning(model_dbow_df123,test_tagged)

In [487]:
logreg(X_df_train,y_df_train,X_test,y_test)

Testing accuracy 0.656140350877193
Testing F1 score: 0.6294786329098112


In [402]:
lr=logreg_multi(X_train,y_train,X_test,y_test)

训练集的准确率：0.998
测试集的准确率：0.000
模型正确率：0.000


In [449]:
# define weight hyperparameter
w = [{0:1898,1:1898,2:1},{0:2000,1:2000,2:1}, {0:3000,1:3000,2:1}, 
     {0:1500,1:1500,2:1}, {0:1000,1:1000,2:1}, {0:1900,1:1000,2:1}, {0:1900,1:1900,2:10}, 
     {0:1900,1:1900,2:100}, {0:1900,1:1900,2:1000}, {0:1900,1:1900,2:10000}, {0:3000,1:3000,2:1000}, 
     {0:3000,1:3000,2:100}, {0:3000,1:3000,2:10}, {0:2000,1:2000,2:10}, 
     {0:2000,1:2000,2:100}, {0:1000,1:1000,2:10}, {0:1000,1:1000,2:100}, 
     {0:1000,1:1000,2:100}, {0:1000,1:1000,2:1000}, {0:500,1:500,2:1000} ]
hyperparam_grid = {"class_weight": w}

# define model
lg3 = LogisticRegression(random_state=13,multi_class='multinomial')
# define evaluation procedure
grid = GridSearchCV(lg3,hyperparam_grid,scoring="f1_macro", cv=100, n_jobs=-1, refit=True)
grid.fit(X_train,y_train)
print(f'Best score: {grid.best_score_} with param: {grid.best_params_}')

/home/yanniyang/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best score: 0.3334204708503012 with param: {'class_weight': {0: 1900, 1: 1900, 2: 10}}


/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [451]:
# define hyperparameters
w = [{0:1898,1:1898,2:1},{0:2000,1:2000,2:1}, {0:3000,1:3000,2:1}, 
     {0:1500,1:1500,2:1}, {0:1000,1:1000,2:1}, {0:1900,1:1000,2:1}, {0:1900,1:1900,2:10}, 
     {0:1900,1:1900,2:100}, {0:1900,1:1900,2:1000}, {0:1900,1:1900,2:10000}, {0:3000,1:3000,2:1000}, 
     {0:3000,1:3000,2:100}, {0:3000,1:3000,2:10}, {0:2000,1:2000,2:10}, 
     {0:2000,1:2000,2:100}, {0:1000,1:1000,2:10}, {0:1000,1:1000,2:100}, 
     {0:1000,1:1000,2:100}, {0:1000,1:1000,2:1000}, {0:500,1:500,2:1000} ]
crange = np.arange(0.5, 20.0, 0.5)
hyperparam_grid = {"class_weight": w
                   ,"penalty": ["l1", "l2"]
                   ,"C": crange
                   ,"fit_intercept": [True, False]  }
# logistic model classifier
lg4 = LogisticRegression(random_state=13,multi_class='multinomial')
# define evaluation procedure
grid = GridSearchCV(lg4,hyperparam_grid,scoring="f1_macro", cv=100, n_jobs=-1, refit=True)
grid.fit(X_train,y_train)
print(f'Best score: {grid.best_score_} with param: {grid.best_params_}')

/home/yanniyang/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [450]:
w={0:776512,1:776512,2:409}
#put the content in function here to see the detail message
lr = LogisticRegression(penalty='l2',solver='newton-cg',multi_class='multinomial',class_weight=w)
lr.fit(X_train,y_train)

print("训练集的准确率：%.3f" %lr.score(X_train, y_train))
print("测试集的准确率：%.3f" %lr.score(X_test, y_test))

y_pred = lr.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred) #错误率，也就是np.average(y_test==y_hat)
print("模型正确率：%.3f" %accuracy)
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
#print(f'Area Under Curve_weighted: {roc_auc_score(y_test, y_pred,average=weighted)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')


训练集的准确率：0.994
测试集的准确率：0.004
模型正确率：0.004
Area Under Curve: 0.49507389162561577
Recall score: [0.         0.00492611 0.        ]


/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [417]:
index_list=[]
for i in range(0,len(y_hat)):
    if y_hat[i] != y_test[i]:
        index_list.append(i)
    else:
        continue
false_det=[]
for j in range(0,len(index_list)):
    false_det.append(X_test[index_list[j]])
    

In [394]:
len(false_det)

26

In [496]:
"""
from imblearn import over_sampling 
ros = over_sampling.RandomOverSampler(random_state=0)
X_resampled,y_resampled = ros.fit_resample(X_train,y_train)

from collections import Counter
print(sorted(Counter(y_resampled).items()))


X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
"""
from imblearn.over_sampling import SMOTE, ADASYN,BorderlineSMOTE

X_resampled, y_resampled = BorderlineSMOTE().fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))



[(0, 776512), (1, 776512), (2, 776512)]


In [500]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
X_under_sampled, y_under_resampled = cc.fit_resample(X_train, y_train)
print(sorted(Counter(y_under_resampled).items()))


[(0, 409), (1, 409), (2, 409)]


In [501]:
#after random over sampling--accuracy = 33% and the AUC is around the 52%
#after SMOTE oversampling-- accuracy =31.2%, AUC = 49.9%
#after ASASYN oversampling-- accuracy =31.2% AUC=58.6%
#ASASYN+weight=1900/1900/1, accuracy=50.9% and AUC=0.4625
#put the content in function here to see the detail message
#using Borderling SMOTE and the accuracy =0.07 while AUC=0.4995 
w={0:776512,1:776512,2:409}
lr = LogisticRegression(penalty='l2',solver='newton-cg',multi_class='multinomial',class_weight='balanced')
lr.fit(X_resampled,y_resampled)

print("训练集的准确率：%.3f" %lr.score(X_resampled, y_resampled))
print("测试集的准确率：%.3f" %lr.score(X_test, y_test))

y_pred = lr.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred) #错误率，也就是np.average(y_test==y_hat)
print("模型正确率：%.3f" %accuracy)
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
#print(f'Area Under Curve_weighted: {roc_auc_score(y_test, y_pred,average=weighted)}')
print(f'Recall score: {recall_score(y_test,y_pred,average=None)}')

/home/yanniyang/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


训练集的准确率：0.962
测试集的准确率：0.070
模型正确率：0.070
Area Under Curve: 0.4995194040610357
Recall score: [0.         0.09852217 0.        ]


/home/yanniyang/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
###test only use two symptom csv file and the newly trained models to see the accuracy 

